---
#### Overview


If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

We are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



In [7]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize

distinct_tokens={}
distinct_pairs={}

# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        for token in t:
            distinct_tokens[token] = distinct_tokens.get(token, 0)
            for t2 in t:
                if token != t2:
                    distinct_pairs[(token,t2)] = distinct_pairs.get((token,t2), 0)

print("distinct tokens: ", len(distinct_tokens))
print("distinct token pairs: ", len(distinct_pairs))
# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

distinct tokens:  25975
distinct token pairs:  1969760


In [9]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

tokens={}
token_pairs={}
totalLines = 0
# Now, let's tokenize Shakespeare's plays


with open('Shakespeare.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        totalLines+=1
        # find distinct tokens
        set_t = set(t)
        # for each distinct token in the set, add 1 to its occurence
        for token in set_t:
            tokens[token] = tokens.get(token, 0) + 1
            # for each distinct token pair in the set, add 1 to its occurence
            for t2 in set_t:
                if token != t2:
                    token_pairs[token] = token_pairs.get(token, {})
                    token_pairs[token][(token,t2)] = token_pairs[token].get((token,t2),0)+1

# helper PMI function
def pmi(v,pq):
    p = tokens[v[0]]/totalLines
    q = tokens[v[1]]/totalLines
    return log((pq/totalLines)/(p*q), 10)


while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        number = 0
        top5 ={}
        if q_tokens[0] in tokens:
            number = tokens[q_tokens[0]]
            # for each token pair associated with token, filter those that are above the threshold
            filtered_dict = {k:v for (k,v) in token_pairs[q_tokens[0]].items() if v >= threshold}
            # create another dictionary for PMI
            pmi_dict = {k:pmi(k,v) for (k,v) in filtered_dict.items()}
            # top 5 largest PMI
            top5 = dict(sorted(pmi_dict.items(),key=lambda item: item[1], reverse=True)[:5])
        print("  n({0}) = {1}".format(q_tokens[0], number ))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        if len(top5) == 0:
            print(" None")
        for item in top5:
            print(" n({0},{1}) = {2},  PMI({0},{1}) = {3:.3f}".format(q_tokens[0], item[1], filtered_dict[item], top5[item])) 
    elif len(q_tokens) == 2:
        # PMI calculation
        if (q_tokens[0] in tokens) and (q_tokens[1] in tokens):
            p = tokens[q_tokens[0]]/totalLines
            q = tokens[q_tokens[1]]/totalLines
            if (q_tokens[0],q_tokens[1]) in token_pairs[q_tokens[0]]:
                pq = token_pairs[q_tokens[0]][(q_tokens[0],q_tokens[1])]
                PMI = log((pq/totalLines)/(p*q), 10)  
                # prints
                print("  n({0},{1}) = {2:.3f}".format(q_tokens[0],q_tokens[1], pq))
                print("  PMI({0},{1}) = {2:.3f}".format(q_tokens[0],q_tokens[1], PMI))
            else:
                print (" token pair does not exist")
        else:
            print(" one of the tokens do not exist")
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")


Input 1 or 2 space-separated tokens (return to quit): love
Input a positive integer frequency threshold: 40
  n(love) = 2020
  high PMI tokens with respect to love (threshold: 40):
 n(love,true) = 48,  PMI(love,true) = 0.561
 n(love,thee) = 126,  PMI(love,thee) = 0.399
 n(love,her) = 137,  PMI(love,her) = 0.381
 n(love,do) = 130,  PMI(love,do) = 0.337
 n(love,if) = 122,  PMI(love,if) = 0.331
Input 1 or 2 space-separated tokens (return to quit): love hate
  n(love,hate) = 34.000
  PMI(love,hate) = 1.074
Input 1 or 2 space-separated tokens (return to quit): lalalla
Input a positive integer frequency threshold: 60
  n(lalalla) = 0
  high PMI tokens with respect to lalalla (threshold: 60):
 None
Input 1 or 2 space-separated tokens (return to quit): cool you
  n(cool,you) = 1.000
  PMI(cool,you) = -0.490
Input 1 or 2 space-separated tokens (return to quit): lulu lala
 one of the tokens do not exist
Input 1 or 2 space-separated tokens (return to quit): you lala
 one of the tokens do not exis